<a href="https://colab.research.google.com/github/Mayank-004/Boston-Crime/blob/main/BostonCRIME_Pyspark_Randomforest.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [3]:
!pip install pyspark



     |████████████████████████████████| 212.3MB 68kB/s 
     |████████████████████████████████| 204kB 39.0MB/s 
  Created wheel for pyspark: filename=pyspark-3.1.1-py2.py3-none-any.whl size=212767604 sha256=3907545e3d05fd8ca9a71f2ea7e2012f760bae8dc265ad9cd449d962908b8f38
  Stored in directory: /root/.cache/pip/wheels/0b/90/c0/01de724414ef122bd05f056541fb6a0ecf47c7ca655f8b3c0f
Successfully built pyspark


In [4]:
import pyspark
import pandas as pd
#from pyspark.sql import SparkSession
#from pyspark.sql.types import StructType,StructField, StringType, IntegerType 
#from pyspark.sql.types import ArrayType, DoubleType, BooleanType
#from pyspark.sql.functions import col,array_contains
#fdf = spark.read.load("crime-incident-reports-2015.csv",format="csv", sep=",", inferSchema="true", header="true")

In [ ]:

from pyspark import SparkConf, SparkContext
from pyspark.sql import SQLContext
sc= SparkContext()
sqlContext = SQLContext(sc)

In [28]:
df1 = sqlContext.read.csv("/content/drive/MyDrive/Boston Crime Data/crime-incident-reports-2015.csv",header=True)
df2 = sqlContext.read.csv("/content/drive/MyDrive/Boston Crime Data/crime-incident-reports-2016.csv",header=True)
df3 = sqlContext.read.csv("/content/drive/MyDrive/Boston Crime Data/crime-incident-reports-2017.csv",header=True)
df4 = sqlContext.read.csv("/content/drive/MyDrive/Boston Crime Data/crime-incident-reports-2018.csv",header=True)
df5 = sqlContext.read.csv("/content/drive/MyDrive/Boston Crime Data/crime-incident-reports-2019.csv",header=True)
df6 = sqlContext.read.csv("/content/drive/MyDrive/Boston Crime Data/crime-incident-reports-2020.csv",header=True)
df6 = sqlContext.read.csv("/content/drive/MyDrive/Boston Crime Data/crime-incident-reports-2021.csv",header=True)
union_data = df1.unionAll(df2).unionAll(df3).unionAll(df4)


In [16]:
union_data.show()

+---------------+------------+-------------------+--------------------+--------+--------------+--------+-------------------+----+-----+-----------+----+----------+-------------+-----------+------------+--------------------+
|INCIDENT_NUMBER|OFFENSE_CODE| OFFENSE_CODE_GROUP| OFFENSE_DESCRIPTION|DISTRICT|REPORTING_AREA|SHOOTING|   OCCURRED_ON_DATE|YEAR|MONTH|DAY_OF_WEEK|HOUR|  UCR_PART|       STREET|        Lat|        Long|            Location|
+---------------+------------+-------------------+--------------------+--------+--------------+--------+-------------------+----+-----+-----------+----+----------+-------------+-----------+------------+--------------------+
|     I192068249|       02647|              Other|THREATS TO DO BOD...|      B2|           280|    null|2015-08-28 10:20:00|2015|    8|     Friday|  10|  Part Two|WASHINGTON ST|42.33011862|-71.08425106|(42.33011862, -71...|
|     I192061894|       01106|   Confidence Games|FRAUD - CREDIT CA...|     C11|           356|    null|

In [17]:
union_data.describe().toPandas().transpose()


,0,1,2,3,4
summary,count,mean,stddev,min,max
INCIDENT_NUMBER,353253,None,None,I010370257-00,I192077559
OFFENSE_CODE,353253,2318.6886650644155,1185.3273469951023,00111,3201
OFFENSE_CODE_GROUP,353253,None,None,Aggravated Assault,Warrant Arrests
OFFENSE_DESCRIPTION,353253,None,None,"A&B HANDS, FEET, ETC. - MED. ATTENTION REQ.",WEAPON - OTHER - OTHER VIOLATION
DISTRICT,351426,None,None,A1,E5
REPORTING_AREA,353253,384.54719403941095,241.8431942724011,,99
SHOOTING,1455,None,None,Y,Y
OCCURRED_ON_DATE,353253,None,None,2015-06-15 00:00:00,2018-12-31 23:45:00
YEAR,353253,2016.6950174520812,1.036961986645097,2015,2018


In [26]:
columns_to_drop = ['INCIDENT_NUMBER', 'OFFENSE_CODE','OFFENSE_CODE_GROUP','OFFENSE_DESCRIPTION','SHOOTING','OCCURRED_ON_DATE','Location']

union_data_clean = union_data.drop(*columns_to_drop)

In [20]:
union_data_clean=union_data_clean.na.drop()


In [270]:
union_data_clean=union_data_clean.fillna(0)


In [27]:
union_data_clean.show()

+--------+--------------+----+-----+-----------+----+----------+-------------+-----------+------------+
|DISTRICT|REPORTING_AREA|YEAR|MONTH|DAY_OF_WEEK|HOUR|  UCR_PART|       STREET|        Lat|        Long|
+--------+--------------+----+-----+-----------+----+----------+-------------+-----------+------------+
|      B2|           280|2015|    8|     Friday|  10|  Part Two|WASHINGTON ST|42.33011862|-71.08425106|
|     C11|           356|2015|    8|   Thursday|   0|  Part Two|   CHARLES ST|42.30060543|-71.06126785|
|      A1|           172|2015|   11|     Monday|  12|  Part Two|    ALBANY ST|42.33428841|-71.07239518|
|     E18|           525|2015|    7|     Friday|  10|  Part Two|   WINGATE RD| 42.2370095|-71.12956606|
|      D4|           159|2015|   12|    Tuesday|  12|  Part Two|     UPTON ST|42.34243222|-71.07225766|
|     C11|           336|2015|   12|     Friday|   0|  Part Two|    NORTON ST|42.30626521|-71.06864556|
|     E18|           478|2015|   12|    Tuesday|  12|  Part Two|

In [21]:
union_data_clean = union_data_clean.withColumn("REPORTING_AREA", union_data_clean['REPORTING_AREA'].cast('float'))
union_data_clean = union_data_clean.withColumn("MONTH", union_data_clean['MONTH'].cast('float'))
union_data_clean = union_data_clean.withColumn("HOUR", union_data_clean['HOUR'].cast('float'))
union_data_clean = union_data_clean.withColumn("Lat", union_data_clean['Lat'].cast('float'))
union_data_clean = union_data_clean.withColumn("Long", union_data_clean['Long'].cast('float'))
#union_data_clean = union_data_clean.withColumn("REPORTING_AREA", union_data_clean.call_time.cast('float'))


In [22]:

from pyspark.ml.feature import VectorAssembler
from pyspark.ml.feature import StringIndexer

from pyspark.ml.feature import StringIndexer
from pyspark.ml import Pipeline

indexers = [StringIndexer(inputCol="DISTRICT", outputCol="DISTRICTIndex") ,StringIndexer(inputCol="DAY_OF_WEEK", outputCol="DAY_OF_WEEKIndex") , StringIndexer(inputCol="UCR_PART", outputCol="UCR_PARTIndex"),StringIndexer(inputCol="STREET", outputCol="STREETIndex"),StringIndexer(inputCol="YEAR", outputCol="YEARIndex")]


pipeline = Pipeline(stages=indexers)
union_data_clean = pipeline.fit(union_data_clean).transform(union_data_clean)

#union_data_clean=indexers.setHandleInvalid("skip").fit(union_data_clean).transform(union_data_clean)


vectorAssembler = VectorAssembler(inputCols = ['DISTRICTIndex','REPORTING_AREA','YEARIndex','MONTH','DAY_OF_WEEKIndex','HOUR','STREETIndex','Lat','Long'], outputCol = 'features')
#vectorAssembler=vectorAssembler.transform(union_data_clean.na.drop)

vunion_data = vectorAssembler.transform(union_data_clean)


vunion_data_f = vunion_data.select(['features', 'UCR_PARTIndex'])



In [23]:
vunion_data_f.show()

+--------------------+-------------+
|            features|UCR_PARTIndex|
+--------------------+-------------+
|[0.0,280.0,3.0,8....|          1.0|
|[1.0,356.0,3.0,8....|          1.0|
|[4.0,172.0,3.0,11...|          1.0|
|[7.0,525.0,3.0,7....|          1.0|
|[2.0,159.0,3.0,12...|          1.0|
|[1.0,336.0,3.0,12...|          1.0|
|[7.0,478.0,3.0,12...|          1.0|
|[0.0,258.0,3.0,9....|          1.0|
|[3.0,455.0,3.0,7....|          1.0|
|[0.0,326.0,3.0,7....|          1.0|
|[0.0,326.0,3.0,7....|          0.0|
|[0.0,326.0,3.0,7....|          1.0|
|[2.0,149.0,3.0,12...|          1.0|
|[2.0,149.0,3.0,12...|          2.0|
|[7.0,486.0,3.0,7....|          1.0|
|[2.0,129.0,3.0,10...|          1.0|
|[6.0,791.0,3.0,9....|          1.0|
|[1.0,366.0,3.0,8....|          0.0|
|[1.0,450.0,3.0,10...|          1.0|
|[1.0,352.0,3.0,8....|          1.0|
+--------------------+-------------+
only showing top 20 rows



In [148]:
'''
splits = vunion_data_f.randomSplit([0.9, 0.1])
train_df = splits[0]
test_df = splits[1]
'''


In [133]:
'''
from pyspark.sql.functions import percent_rank
from pyspark.sql import Window

vunion_data_f = vunion_data_f.withColumn("rank", percent_rank().over(Window.partitionBy().orderBy("UCR_PARTIndex")))
train_df = vunion_data_f.where("rank <= .8").drop("rank")
#train_df.show()
test_df = vunion_data_f.where("rank > .8").drop("rank")
#test_df.show()

'''

In [ ]:
'''
from pyspark.ml.regression import DecisionTreeRegressor
from pyspark.ml.tuning import CrossValidator, ParamGridBuilder
from pyspark.ml.feature import VectorAssembler

#vectorAssembler = VectorAssembler(inputCols = ['Cylinders','ManufactureIndex','ModelIndex','OriginIndex','Displacement','Weight'], outputCol = 'features')
#vunion_data = vectorAssembler.transform(vunion_data)

dt = DecisionTreeRegressor(featuresCol ='features', labelCol = 'UCR_PARTIndex')




dt_model = dt.fit(train_df)

dt_predictions_train = dt_model.transform(train_df)

dt_predictions_test = dt_model.transform(test_df)
dt_evaluator = RegressionEvaluator(labelCol="Horsepower", predictionCol="prediction", metricName="rmse")
rmse_train = dt_evaluator.evaluate(dt_predictions_train)
rmse_test = dt_evaluator.evaluate(dt_predictions_test)

print("Root Mean Squared Error (RMSE) on train data = %g" % rmse_train)
print("Root Mean Squared Error (RMSE) on test data = %g" % rmse_test)
'''




In [24]:
from pyspark.ml import Pipeline
from pyspark.ml.classification import DecisionTreeClassifier
from pyspark.ml.feature import StringIndexer, VectorIndexer
from pyspark.ml.evaluation import MulticlassClassificationEvaluator


(trainingData, testData) = vunion_data_f.randomSplit([0.7, 0.3])

# Train a DecisionTree model.
#pipeline = Pipeline().setStages(indexers + [vectorAssembler,dt])

dt = DecisionTreeClassifier(labelCol="UCR_PARTIndex", featuresCol="features")

# Chain indexers and tree in a Pipeline
#pipeline = Pipeline(stages=['UCR_PARTIndex', 'features', dt])

# Train model.  This also runs the indexers.
model = dt.fit(trainingData)

# Make predictions.
predictions = model.transform(testData)

# Select example rows to display.
predictions.select("prediction", "UCR_PARTIndex", "features").show(5)

# Select (prediction, true label) and compute test error
evaluator = MulticlassClassificationEvaluator(
    labelCol="UCR_PARTIndex", predictionCol="prediction", metricName="accuracy")
accuracy = evaluator.evaluate(predictions)
print("Test Error = %g " % (1.0 - accuracy))

treeModel = model.stages[2]
# summary only
print(treeModel)

Py4JJavaError: ignored

In [ ]:
 pipeline = Pipeline().setStages(indexer + [assembler,dtc_model])